In [ ]:
#@title ライブラリーの利用
!git clone https://github.com/Michi-123/whiteGPT

Cloning into 'whiteGPT'...
remote: Enumerating objects: 1299, done.
remote: Counting objects: 100% (135/135), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 1299 (delta 84), reused 88 (delta 40), pack-reused 1164 (from 1)
Receiving objects: 100% (1299/1299), 136.87 MiB | 26.87 MiB/s, done.
Resolving deltas: 100% (743/743), done.


https://en.wikipedia.org/wiki/Word2vec

In [5]:
#@title ライブラリーの読み込み
import sys
sys.path.append("/content/whiteGPT")

import random
import torch
import torch.nn as nn
import torch.optim as optim
from whiteGPT import word2vec
from whiteGPT import CBOW
from whiteGPT import Vocab

In [3]:
# @title 学習用のテキストデータ
corpus_list = [
    "this animal is cat.",
    "the quick brown fox jumps over the lazy dog.",
    "dog and cat are animal.",
    "I love dogs and cats.",
    "the dog sat on the rug.",
    "cats are independent animals. ",
    "foxes are wild animals.",
    "The cat prowled through the moonlit garden.",
    "Dogs eagerly awaited their owner's return at the doorstep.",
    "A cat's purr filled the room with comfort.",
    "The dog wagged its tail in excitement.",
    "Cats gracefully leaped from rooftop to rooftop.",

    "Walking down the street, I spotted a stray dog searching for scraps.",
    "The cat stretched lazily in the warmth of the sunbeam.",
    "Dogs barked joyfully in the park.",
    "A sleek black cat slinked along the fence.",
    "The old dog snoozed contentedly by the fireplace.",
    "Cats darted through the alleyways, chasing shadows.",
    "A fluffy white cat napped peacefully on the windowsill.",

    "Dogs are furry friends who love to play fetch and cuddle with you.",
    "Cats are soft and independent pets that enjoy lounging in sunny spots.",
    "Dogs wag their tails when they're happy and bark to say hello.",
    "Cats purr when they're content and love to curl up in your lap.",

    "Fido loves going for car rides, he always sticks his head out the window with a goofy grin.",
    "the old dog hobbled over to greet us, his tail thumping gently against the floor.",
    "the cat perched regally on the windowsill, surveying its outdoor kingdom. ",
    "the cat stalked a dust bunny across the floor, pouncing with laser focus. ",
    "the neighbor's cat, notorious for its thievery, snuck into our yard and made off with a shiny red ball of yarn. ",
    "curled up with a good book, I felt a soft nudge – my cat, wanting some attention, was rubbing against my leg. ",
    "the therapy dog, with its gentle demeanor, brought a wave of calm to the anxious patients in the waiting room.",
    "despite their different personalities, the dog and cat often napped curled up together. ",
    "we need to buy more cat food, Whiskers seems to be inhaling everything in the bow. ",
    "despite being a scaredy cat, Luna the ginger bravely explored every corner of the new house. ",
    "Max the dog spent all afternoon digging a hole in the backyard, much to the gardener's dismay. ",
    "the smell of freshly baked cookies lured the cat out from its hiding spot. ",
    "during thunderstorms, Milo the dog would huddle under the bed, trembling uncontrollably. ",
    "we adopted a pair of playful kittens, and now our living room is a whirlwind of fur and feathery toys. ",
    "every morning, the rooster crows and the dog barks, creating a chaotic symphony to wake up the household. "
]

In [6]:
# @title テスト用のテキストデータ
test_corpus_list = [
    "this animal is dog ",
    "the old cat hobbled ",
    "the dog perched regally ",
    "quick brown cat jumps ",
    "cat and dog often ",
    "the dog out from ",
    "Fido the cat would ",
    "Luna the cat spent ",
    "neighbor's dog ",
    "cat stalked a dust ",
    "buy more cat food",
    "cat and the dog barks",
]

# CBOWモデルの学習

In [7]:
# @title ハイパー・パラメータの設定
# @markdown 埋め込みの次元の数
embedding_dim = 16 #@param{type:'integer'}
# @markdown エポック数
num_epoch = 300 #@param{type:'integer'}
# @markdown 学習係数（学習率）
learning_rate = 0.001 #@param{type:'number'}
# @markdown 入力枠
window_size = 3 #@param{type:'integer'}
# word2vec.window_size = window_size

In [18]:
# @title インスタンス化
# 学習しやすいデータに加工します
pad = '<PAD>' * window_size
train_corpus = pad.join(corpus_list)
train_corpus = word2vec.modify(corpus_list, window_size) # 追加コード

# (バグ修正用の追加コード)
import re
train_corpus = train_corpus.lower()
train_corpus = re.findall(r'\w+|[^\w\s]', train_corpus)
train_corpus = ' '.join(train_corpus)

# Dataset
vocab = Vocab(train_corpus)
train_dataset = word2vec.TextDataset(vocab, train_corpus, window_size)
train_dataloader = word2vec.DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataset = word2vec.TextDataset(vocab, "", window_size)
# 検証しやすいデータを設定します
test_dataset.test_corpus(test_corpus_list)
# 損失関数の計算
criterion = nn.CrossEntropyLoss()
# モデルのインスタンス化
model = word2vec.CBOW(vocab.vocab_size, embedding_dim)
# 最適化モジュール
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

In [17]:
# debug
train_iter = iter(train_dataloader)
next(train_iter)

{'source': tensor([[150, 234, 225],
         [122, 225,  39],
         [ 69,  24,  98],
         [  0,   0,   0]]),
 'target': tensor([100,   8,  95,   0])}

In [21]:
# @title 学習の実行
for epoch in range(num_epoch):
    total_loss = 0
    running_loss = 0
    for batch in train_dataloader:

        # 入力データと教師データの取得
        context_indices, target_index = batch["source"],batch["target"]
        # 推論
        output = model(context_indices)
        # 損失計算
        loss = criterion(output,target_index)
        # 勾配の初期化
        optimizer.zero_grad()
        # 勾配の計算
        loss.backward()
        # 学習パラメータ（重み）の更新
        optimizer.step()
        # 損失の累計
        running_loss += loss.item()

    # 損失の平均
    total_loss = running_loss / len(train_dataloader)

    # n回に一度の処理
    if epoch % 10 == 0:
        # ログの出力
        print(f"Epoch{epoch},Loss{total_loss}")
        # テスト
        test_dataset.test(model)

Epoch0,Loss5.230410956419432
buy more cat : food : street
Epoch10,Loss2.7451234785123515
cat and dog : often : ,
Epoch20,Loss2.18870492800712
cat and dog : often : and
Epoch30,Loss1.7402200617540915
the dog out : from : ,
Epoch40,Loss1.383275784050616
the dog perched : regally : ,
Epoch50,Loss1.11189345890083
and the dog : barks : with
Epoch60,Loss0.9057013564877427
buy more cat : food : food
Epoch70,Loss0.7526102969422936
cat and dog : often : often
Epoch80,Loss0.6355293617368891
luna the cat : spent : regally
Epoch90,Loss0.5494046083275372
cat stalked a : dust : dust
Epoch100,Loss0.4859420824527311
cat stalked a : dust : dust
Epoch110,Loss0.4383626444480167
fido the cat : would : room
Epoch120,Loss0.40371875365291576
cat and dog : often : often
Epoch130,Loss0.3792909561184038
cat stalked a : dust : dust
Epoch140,Loss0.3597011820708688
this animal is : dog : cat
Epoch150,Loss0.34716247368823344
and the dog : barks : cat
Epoch160,Loss0.33724120641104843
cat and dog : often : often
Epoc

# 評価

In [29]:
# 次の単語を予測
test_dataset.test(model)

the old cat : hobbled : napped


In [30]:
# 重みの表示
model.embeddings.weight

Parameter containing:
tensor([[ 1.4509,  0.0065,  0.3409,  ..., -1.9526,  0.6899, -0.4549],
        [ 0.1846,  1.3079,  0.7179,  ..., -0.1679,  1.1737,  2.0233],
        [-0.3015,  0.0137, -0.1076,  ...,  0.2195, -2.6794,  0.1836],
        ...,
        [-2.7816, -2.5364,  1.8666,  ..., -4.7798,  4.5953, -3.3245],
        [-3.3377, -4.7171,  5.0621,  ..., -3.8381,  1.2953, -3.8615],
        [ 2.8610, -1.8987,  4.7440,  ..., -2.8966, -1.4770, -5.6472]],
       requires_grad=True)